In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Jankomo001"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),

    html.Div(
        style={
            'display': 'flex',
            'align-items': 'center',    
            'justify-content': 'center',    
            'gap': '10px'                   
        },
        children=[
            html.A(
                href="https://www.snhu.edu",
                target="_blank",
                children=html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': '120px'}  
                )
            ),
            html.Div(
                children=[
                    html.B(html.H1('CS-340 Dashboard')),
                    html.B(html.H2('Thomas Davis'))
                ],
                style={
                    'textAlign': 'left' 
                }
            )
        ]
    ),
    html.Hr(),
    html.Div(
      # Creates selectable radio buttons for filters
      dcc.RadioItems(
          id='filter-type-radio',
          options=[
              {'label': 'Water Rescue', 'value': 'water'},
              {'label': 'Mountain Rescue', 'value': 'mountain'},
              {'label': 'Disaster', 'value': 'disaster'},
              {'label': 'Reset', 'value': 'reset'}
          ],
          value='reset', 
          labelStyle={'display': 'inline-block', 'margin-right': '20px'} 
      ),
      style={'textAlign': 'center', 'padding': '10px'}  
    ),
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
            dash_table.DataTable(
                id='datatable-id',
                columns=[{"name": i, 
                          "id": i, 
                          "deletable": False, 
                          "selectable": True} 
                         for i in df.columns],
                data=df.to_dict('records'),
                editable=False,
                filter_action="native",                               
                sort_action="native",
                sort_mode="multi",
                column_selectable=False,
                row_selectable="single",
                row_deletable=False,
                selected_columns=[],
                selected_rows=[],
                page_action="native",
                page_current=0,
                page_size=10,
                style_table={
                    'height': '400px', 
                    'overflowY': 'auto',
                    'border-bottom': '1px solid #000'
            }
        )
    ], style={'height': '410px'}),
                         
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here
    
# Sets the size and placement of the graph and map
    html.Div(
        style={'display': 'flex', 
               'justify-content': 'center', 
               'align-items': 'flex-start', 
               'gap': '10px'},
        children=[
            html.Div(
                id='graph-id',
                style={
                    'flex': '0 0 48%',
                    'padding': '5px',
                    'display' : 'flex',
                    'justify-content' : 'center',
                    'align-items' : 'center',
                    'padding-bottom' : '80px'
                }
            ),
            html.Div(
                id='map-id',
                style={
                    'flex': '0 0 48%',
                    'padding': '5px',
                    'display' : 'flex',
                    'justify-content' : 'center',
                    'align-items' : 'center',
                    'padding-top' : '40px'
                }
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type-radio', 'value')]
)


def update_dashboard(filter_type):

    # Filters the data using query
    if filter_type == 'water':
        # Water Rescue:
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        # Mountain Rescue:
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", 
            "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        # Disaster
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else: 
        query = {}

    # Read from the database with query
    filtered_df = pd.DataFrame.from_records(db.read(query))
    
    # Drop the '_id' column 
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)

    return filtered_df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table

@app.callback(Output('graph-id', "children"),
              [Input('datatable-id', "derived_virtual_data")])

# Show the data in a pie chart
def update_graphs(viewData):
    
    
    if viewData is None or len(viewData) == 0:
        return []
    
    df = pd.DataFrame.from_dict(viewData)

    breed_counts = df['breed'].value_counts()
    
    breed_df = breed_counts.reset_index()
    breed_df.columns = ['breed', 'count']
    
    fig = px.pie(breed_df,
        names='breed',
        values='count',
        hover_data=['count'], 
        labels={'count': 'Number of Animals', 'breed': 'Breed'},
    )
    
    fig.update_traces(textposition='inside', textinfo='percent+label')
    
    fig.update_layout(
        legend=dict(
        y=0.5
    ),
    margin=dict(t=20, b=20, l=20, r=20),
    autosize=False
)

    return [dcc.Graph(figure=fig,style={'height': '700px', 'width': '700px'})]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')
              ])

def update_styles(selected_columns):
    
    # Added to fix a NoneType error
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
              Input('datatable-id', "derived_virtual_selected_rows")])


def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]  
    
    # Get the coordinates from the selected row
    lat = dff.iloc[row, 13]
    lon = dff.iloc[row, 14]
    
    return [
        dl.Map(
            style={'width': '75%', 'height': '400px'},
            center=[lat, lon], 
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(dff.iloc[row, 4]), 
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row, 9])  
                    ])
                ])
            ]
        )
    ]




app.run_server(debug=True)


Dash app running on http://127.0.0.1:12054/
